# Travel Agent AI

This notebook creates a basic AI travel agent using OpenAI's `gpt-4o-mini` via GitHub Models. It suggests random vacation destinations and plans trips based on user input.

No `semantic_kernel` here—just raw `openai` and some Python magic. 🫶

## Import the Needed Python Packages

In [1]:
!python --version

Python 3.12.10


In [2]:
!python -c "import pydantic; print(pydantic.__version__)"

2.11.4


In [3]:
import os
import random
from typing import Optional
from openai import AsyncOpenAI
from dotenv import load_dotenv

## Creating the Client

We’re using [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) to access `gpt-4o-mini`. Make sure your `GITHUB_TOKEN` is set in a `.env` file. You can swap `gpt-4o-mini` for another model available on GitHub Models to mix things up.

In [4]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"),
    base_url="https://models.inference.ai.azure.com/",
)

## Creating the Destination Picker

This `DestinationsPlugin` picks random vacation spots, just like the original. No fancy frameworks—just a simple class.

In [5]:
class DestinationsPlugin:
    """A list of random vacation destinations."""

    def __init__(self):
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        self.last_destination = None

    def get_random_destination(self) -> str:
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        destination = random.choice(available_destinations)
        self.last_destination = destination
        return destination

## Setting Up the Agent Logic

We’ll create a function to handle the AI chat, using `openai` directly. It mimics the `TravelAgent` by combining a random destination with user input and a system prompt.

In [6]:
async def run_travel_agent(user_input: str, plugin: DestinationsPlugin) -> str:
    system_prompt = "You are a helpful AI travel agent that plans vacations for customers at random destinations. Use the provided destination to craft a response."
    destination = plugin.get_random_destination()
    prompt = f"{user_input}\nSuggested destination: {destination}"

    response = await client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content

## Running the Agent

Let’s test the agent with a sample input: `Plan me a day trip.` Feel free to change the input to see different responses!

In [7]:
async def main():
    plugin = DestinationsPlugin()
    user_inputs = ["Plan me a day trip."]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        response = await run_travel_agent(user_input, plugin)
        print(f"# TravelAgent: {response}\n")

await main()

# User: Plan me a day trip.

# TravelAgent: Absolutely! Barcelona is a vibrant city filled with stunning architecture, rich history, and delicious cuisine. Here’s a suggested itinerary for your day trip in Barcelona:

### Morning

**8:00 AM - Breakfast at Brunch & Cake**
Start your day with a delicious breakfast at Brunch & Cake, a popular café known for its creative and colorful dishes. Try their signature pancakes or a healthy smoothie bowl to fuel your day.

**9:30 AM - Visit La Sagrada Familia**
After breakfast, head to La Sagrada Familia, the iconic basilica designed by Antoni Gaudí. It's best to book your tickets in advance to avoid long lines. Spend some time exploring the stunning architecture and the intricate details of this UNESCO World Heritage Site.

### Midday

**11:30 AM - Stroll through Park Güell**
Take a short taxi or metro ride to Park Güell, another Gaudí masterpiece. Wander through the whimsical park filled with colorful mosaics, unique sculptures, and panoramic vi